stacking

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

# load dataset
categorical_df = pd.read_csv("categorical_dataset.csv")
np.random.seed(42)
categorical_df["target"] = np.random.choice([0, 1], size=len(categorical_df))
X = categorical_df.drop("target", axis=1)
y = categorical_df["target"]

# preprocessing
categorical_features = X.columns.tolist()
preprocessor = ColumnTransformer([("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features)])

# pipelines for base models
dt_clf = Pipeline([("pre", preprocessor), ("clf", DecisionTreeClassifier())])
rf_clf = Pipeline([("pre", preprocessor), ("clf", RandomForestClassifier())])

# stacking model
stacking_clf = StackingClassifier(estimators=[("dt", dt_clf), ("rf", rf_clf)], final_estimator=LogisticRegression(), passthrough=False)

# split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# train
stacking_clf.fit(X_train, y_train)

# predict
y_pred = stacking_clf.predict(X_test)

# accuracy
print("Stacking Accuracy (Categorical):", accuracy_score(y_test, y_pred))


This code demonstrates how to apply stacking ensemble learning to a categorical dataset. The dataset is loaded from categorical_dataset.csv, and since it lacks a target variable, a synthetic binary target (0 or 1) is created randomly. The independent features (X) are separated from the target (y).

Because the dataset consists of categorical features, preprocessing is required. A ColumnTransformer is used with OneHotEncoder to convert categorical variables into numerical format suitable for machine learning models.

Two base learners are defined: a Decision Tree Classifier and a Random Forest Classifier, each wrapped in a pipeline that includes preprocessing. These models capture different aspects of the data—decision trees focus on splitting rules, while random forests reduce overfitting by combining multiple trees.

A StackingClassifier is then constructed, combining the predictions of these base learners. The final estimator is Logistic Regression, which learns how to best combine the outputs of the decision tree and random forest to make final predictions.

The dataset is split into training and testing sets (80/20). The stacking model is trained (fit) and then tested. Finally, accuracy is calculated with accuracy_score, showing how well the stacking ensemble generalizes on unseen data